In [2]:
import pandas as pd
import numpy as np

# FORECAST: IBES, CRSP, Financial Ratios databases 

In [3]:
periods = ['A1', 'A2', 'Q1', 'Q2', 'Q3']

In [4]:
# Create an empty dictionary to store DataFrames
forecast_data = {}

# Read CSV files into DataFrames and store them in the dictionary
for forecast in periods:
    file_path = f"data/final_data/{forecast}.csv"
    forecast_data[forecast] = pd.read_csv(file_path, index_col=0)
    forecast_data[forecast] .reset_index(inplace=True)



## Select columns needed

In [5]:
# example with Q1
Q1 = forecast_data.get('Q1')

# Setting display options to show all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
display(Q1)

,permno,price,ret,cfacshr,rankdate,Unnamed: 0,ticker,cusip,cname,fpedats,statpers,meanest,actual,anndats_act,fpi,capei,be,bm,evm,pe_exi,pe_inc,ps,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,gprof,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,mktcap,ptb,peg_trailing,divyield,ffi49,adj_actual,adj_meanest
0,10001.0,10.980000,0.022727,1.0,2011-02,446868,GFGC,36720410,GAS NATURAL INC,2010-12-31,2011-02-17,0.36,0.24,2011-04-05,6,20.942035,73.898,0.861905,8.378183,11.934783,11.934783,0.940080,12.786876,0.611523,0.063355,0.166109,0.121093,0.166109,0.101486,0.108372,0.140501,0.105587,0.123586,0.375727,0.105989,0.163055,0.105989,0.187206,0.119967,0.110355,0.770447,0.229553,0.434894,0.229553,0.124592,0.062300,0.207756,0.174685,0.380636,0.302059,2.737154,0.472165,0.607822,0.342965,0.390549,0.172855,0.105065,-0.266984,0.840978,0.297153,0.464880,0.408038,0.868740,3.661616,5.087236,0.341804,0.885140,1.072488,49.736410,11.252175,0.845836,6.948400,10.498439,0.956513,1.241503,32.435307,0.000000,0.0,0.0,-0.008597,86.017316,1.164001,0.588007,0.049180,31.0,0.24,0.36
1,10001.0,11.730000,0.072404,1.0,2011-03,446869,GFGC,36720410,GAS NATURAL INC,2010-12-31,2011-03-17,0.36,0.24,2011-04-05,6,22.372502,73.898,0.861905,8.378183,12.750000,12.750000,1.004293,13.660297,0.611523,0.063355,0.166109,0.121093,0.166109,0.101486,0.108372,0.140501,0.105587,0.123586,0.375727,0.105989,0.163055,0.105989,0.187206,0.119967,0.110355,0.770447,0.229553,0.434894,0.229553,0.124592,0.062300,0.207756,0.174685,0.380636,0.302059,2.737154,0.472165,0.607822,0.342965,0.390549,0.172855,0.105065,-0.266984,0.840978,0.297153,0.464880,0.408038,0.868740,3.661616,5.087236,0.341804,0.885140,1.072488,49.736410,11.252175,0.845836,6.948400,10.498439,0.956513,1.241503,32.435307,0.000000,0.0,0.0,-0.008597,91.892816,1.243509,0.628172,0.046036,31.0,0.24,0.36
2,10001.0,11.230000,-0.038789,1.0,2011-04,446870,GFGC,36720410,GAS NATURAL INC,2011-03-31,2011-04-14,0.48,0.52,2011-05-11,6,22.285564,73.898,0.861905,8.378183,12.206522,12.206522,1.000390,13.607214,0.611523,0.063355,0.166109,0.121093,0.166109,0.101486,0.108372,0.140501,0.105587,0.123586,0.375727,0.105989,0.163055,0.105989,0.187206,0.119967,0.110355,0.770447,0.229553,0.434894,0.229553,0.124592,0.062300,0.207756,0.174685,0.380636,0.302059,2.737154,0.472165,0.607822,0.342965,0.390549,0.172855,0.105065,-0.266984,0.840978,0.297153,0.464880,0.408038,0.868740,3.661616,5.087236,0.341804,0.885140,1.072488,49.736410,11.252175,0.845836,6.948400,10.498439,0.956513,1.241503,32.435307,0.000000,0.0,0.0,-0.008597,91.535726,1.238677,0.601395,0.048085,31.0,0.52,0.48
3,10001.0,11.500000,0.028050,1.0,2011-05,446871,GFGC,36720410,GAS NATURAL INC,2011-03-31,2011-05-19,0.48,0.52,2011-05-11,6,21.126099,77.103,0.806422,7.430561,13.855422,13.855422,0.967173,11.407631,0.582306,0.066128,0.166760,0.122867,0.166760,0.109360,0.109433,0.130567,0.107155,0.115835,0.375727,0.107524,0.102424,0.107524,0.187474,0.116494,0.127548,0.746599,0.253401,0.475361,0.253401,0.090574,0.048282,0.164846,0.197437,0.245241,0.326576,2.560420,0.466930,0.590508,0.357410,0.443451,0.225457,0.105065,-0.224656,0.825950,0.338366,0.487082,0.435053,0.949630,4.207708,5.959960,0.279160,0.680486,0.847891,16.937738,13.235976,0.764861,12.788547,9.324220,1.113325,1.491195,32.435307,0.000000,0.0,0.0,-0.014466,93.736500,1.215731,0.311877,0.046957,31.0,0.52,0.48
4,10001.0,11.550000,0.008261,1.0,2011-06,446875,GFGC,36720410,GAS NATURAL INC,2011-06-30,2011-06-16,0.01,0.04,2011-08-12,6,21.217952,77.103,0.806422,7.430561,13.915663,13.915663,0.971378,11.457229,0.582306,0.066128,0

In [15]:
filtered_forecast_data = {}

for forecast, df in forecast_data.items():
    df['rankdate'] = pd.to_datetime(df['rankdate'])
    numeric_vars = df.select_dtypes(include=['number']).columns
    to_predict = ['adj_actual']

    not_prediction = [ "cfacshr", 'Unnamed: 0', 'actual', 'meanest' ,"fpi"]

    cols_used = set(df[list(numeric_vars)+list(['rankdate'])].columns) - set(not_prediction) 

    vars_rf = {'ipi', 'rgdp', 'rpc', 'unemp'} | set(list(cols_used) + to_predict)


    

    # Filter based on the years
    begin_year = 1986
    last_year = 2023
    
    filtered_df = df[(df['rankdate'].dt.year >= begin_year) & (df['rankdate'].dt.year <= last_year)][cols_used]
    
    filtered_df.set_index(['rankdate', 'permno'], inplace=True)
    # Store the filtered DataFrame in the new dictionary
    filtered_forecast_data[forecast + '_filtered'] = filtered_df
    


/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/1660764093.py:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_df = df[(df['rankdate'].dt.year >= begin_year) & (df['rankdate'].dt.year <= last_year)][cols_used]
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/1660764093.py:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_df = df[(df['rankdate'].dt.year >= begin_year) & (df['rankdate'].dt.year <= last_year)][cols_used]
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/1660764093.py:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_df = df[(df['rankdate'].dt.year >= begin_year) & (df['rankdate'].dt.year <= last_year)][cols_used]
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/1660764093.py:21:

In [16]:
# Accessing and analyzing each DataFrame
for forecast, df in filtered_forecast_data.items():
    
    # Missing values per column
    missing_values = df.isna().sum()
    print(f"{forecast} Missing Values:")
    print(missing_values[missing_values > 0])


A1_filtered Missing Values:
price    13
ret      57
dtype: int64
A2_filtered Missing Values:
price     7
ret      43
dtype: int64
Q1_filtered Missing Values:
price     9
ret      52
dtype: int64
Q2_filtered Missing Values:
price     9
ret      47
dtype: int64
Q3_filtered Missing Values:
price     9
ret      39
dtype: int64


In [28]:
# drop left na
for _, df in filtered_forecast_data.items():
    df.dropna(inplace=True)
    df.reset_index(inplace=True)



/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/2904382798.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/2904382798.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_30268/2904382798.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop

In [29]:
Q1_filtered = filtered_forecast_data.get('Q1_filtered')

In [30]:
Q1_filtered

,rankdate,permno,aftret_eq,be,pretret_earnat,ocf_lct,curr_ratio,debt_invcap,curr_debt,efftax,debt_capital,pay_turn,dltt_be,fcf_ocf,pe_exi,evm,debt_assets,roe,ffi49,ps,quick_ratio,roa,rd_sale,rect_turn,cash_debt,inv_turn,gprof,pe_inc,pretret_noa,dpr,ptpm,roce,gpm,bm,pcf,aftret_invcapx,sale_invcap,ptb,price,rect_act,staff_sale,sale_equity,totdebt_invcap,peg_trailing,lt_ppent,opmbd,ret,int_totdebt,mktcap,debt_at,equity_invcap,intcov,opmad,lt_debt,capital_ratio,divyield,intcov_ratio,accrual,aftret_equity,int_debt,cfm,invt_act,profit_lct,at_turn,cash_ratio,debt_ebitda,adv_sale,sale_nwc,de_ratio,npm,capei,adj_actual,cash_conversion,cash_lt,short_debt,adj_meanest
0,2011-02-01,10001.0,0.105989,73.898,0.119967,0.172855,1.072488,0.229553,0.607822,0.375727,0.408038,10.498439,0.297153,-0.266984,11.934783,8.378183,0.464880,0.105587,31.0,0.940080,0.885140,0.140501,0.000000,6.948400,0.105065,11.252175,0.110355,11.934783,0.187206,0.611523,0.101486,0.123586,0.166109,0.861905,12.786876,0.163055,0.956513,1.164001,10.980000,0.380636,0.0,1.241503,0.434894,0.588007,0.840978,0.166109,0.022727,0.062300,86.017316,0.302059,0.770447,3.661616,0.121093,0.342965,0.229553,0.049180,5.087236,-0.008597,0.105989,0.124592,0.108372,0.174685,0.390549,0.845836,0.341804,2.737154,0.0,32.435307,0.868740,0.063355,20.942035,0.24,49.736410,0.207756,0.472165,0.36
1,2011-03-01,10001.0,0.105989,73.898,0.119967,0.172855,1.072488,0.229553,0.607822,0.375727,0.408038,10.498439,0.297153,-0.266984,12.750000,8.378183,0.464880,0.105587,31.0,1.004293,0.885140,0.140501,0.000000,6.948400,0.105065,11.252175,0.110355,12.750000,0.187206,0.611523,0.101486,0.123586,0.166109,0.861905,13.660297,0.163055,0.956513,1.243509,11.730000,0.380636,0.0,1.241503,0.434894,0.628172,0.840978,0.166109,0.072404,0.062300,91.892816,0.302059,0.770447,3.661616,0.121093,0.342965,0.229553,0.046036,5.087236,-0.008597,0.105989,0.124592,0.108372,0.174685,0.390549,0.845836,0.341804,2.737154,0.0,32.435307,0.868740,0.063355,22.372502,0.24,49.736410,0.207756,0.472165,0.36
2,2011-04-01,10001.0,0.105989,73.898,0.119967,0.172855,1.072488,0.229553,0.607822,0.375727,0.408038,10.498439,0.297153,-0.266984,12.206522,8.378183,0.464880,0.105587,31.0,1.000390,0.885140,0.140501,0.000000,6.948400,0.105065,11.252175,0.110355,12.206522,0.187206,0.611523,0.101486,0.123586,0.166109,0.861905,13.607214,0.163055,0.956513,1.238677,11.230000,0.380636,0.0,1.241503,0.434894,0.601395,0.840978,0.166109,-0.038789,0.062300,91.535726,0.302059,0.770447,3.661616,0.121093,0.342965,0.229553,0.048085,5.087236,-0.008597,0.105989,0.124592,0.108372,0.174685,0.390549,0.845836,0.341804,2.737154,0.0,32.435307,0.868740,0.063355,22.285564,0.52,49.736410,0.207756,0.472165,0.48
3,2011-05-01,10001.0,0.107524,77.103,0.116494,0.225457,0.847891,0.253401,0.590508,0.375727,0.435053,9.324220,0.338366,-0.224656,13.855422,7.430561,0.487082,0.107155,31.0,0.967173,0.680486,0.130567,0.000000,12.788547,0.105065,13.235976,0.127548,13.855422,0.187474,0.582306,0.109360,0.115835,0.166760,0.806422,11.407631,0.102424,1.113325,1.215731,11.500000,0.245241,0.0,1.491195,0.475361,0.311877,0.825950,0.166760,0.028050,0.048282,93.736500,0.326576,0.746599,4.207708,0.122867,0.357410,0.253401,0.046957,5.959960,-0.014466,0.107524,0.090574,0.109433,0.197437,0.443451,0.764861,0.279160,2.560420,0.0,32.435307,0.949630,0.066128,21.126099,0.52,16.937738,0.164846,0.466930,0.48
4,2011-06-01,10001.0,0.107524,77.103,0.116494,0.225457,0.847891,0.253401,0.590508,0.375727,0.435053,9.324220,0.338366,-0.224656,13.915663,7.430561,0.487082,0.107155,31.0,0.971378,0.680486,0.130567,0.000000,12.788547,0.105065,13.235976,0.127548,13.915663,0.187474,0.582306,0.109360,0.115835,0.166760,0.806422,11.457229,0.102424,1.113325,1.221017,11.550000,0.245241,0.0,1.491195,0.475361,0.313233,0.825950,0.166760,0.008261,0.048282,94.144052,0.326576,0.746599,4.207708,0.122867,0.357410,0.253401,0.046753,5.959960,-0.014466,0.107524,0.090574,0.109433,0.197437,0.443451,0.764861,0.279160,2.560420,0.0,32.435307,0.949630,0.066128,21.2179